In [1]:
# Snowpark for Python
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.functions import udf
import snowflake.snowpark.functions as F

# Snowpark ML
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.registry import Registry
from snowflake.ml._internal.utils import identifier

import pandas as pd

In [2]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
#     "authenticator": "externalbrowser", # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
} 

In [3]:
# Make a Snowpark Connection

################################################################################################################
#  You can also use the SnowSQL Client to configure your connection params:
#  https://docs.snowflake.com/en/user-guide/snowsql-install-config.html
#
#  >>> from snowflake.ml.utils import connection_params
#  >>> session = Session.builder.configs(connection_params.SnowflakeLoginOptions()
#  >>> ).create()   
#
#  NOTE: If you have named connection params then specify the connection name
#  Example:
#  
#  >>> session = Session.builder.configs(
#  >>> connection_params.SnowflakeLoginOptions(connection_name='connections.snowml')
#  >>> ).create()
#
#################################################################################################################

# Edit the connection.json before creating the session object below
# Create Snowflake Session object
# connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : ADITYASINGH
Role                        : "ADITYASINGH"
Database                    : "FIRST_DB"
Schema                      : "PUBLIC"
Warehouse                   : "FOSFOR_INSIGHT_WH"
Snowflake version           : 8.20.10
Snowpark for Python version : 1.17.0


In [11]:
data = pd.read_csv('/data/diamonds.csv')
data.show()

AttributeError: 'DataFrame' object has no attribute 'show'

In [10]:
#moving data to snowflake df
diamonds_df = session.create_dataframe(data)
diamonds_df.show()
diamonds_df.drop(["Unnamed: 0"])
diamonds_df.show()

---------------------------------------------------------------------------------------------------------------
|"Unnamed: 0"  |"carat"  |"cut"      |"color"  |"clarity"  |"depth"  |"table"  |"price"  |"x"   |"y"   |"z"   |
---------------------------------------------------------------------------------------------------------------
|1             |0.23     |Ideal      |E        |SI2        |61.5     |55.0     |326      |3.95  |3.98  |2.43  |
|2             |0.21     |Premium    |E        |SI1        |59.8     |61.0     |326      |3.89  |3.84  |2.31  |
|3             |0.23     |Good       |E        |VS1        |56.9     |65.0     |327      |4.05  |4.07  |2.31  |
|4             |0.29     |Premium    |I        |VS2        |62.4     |58.0     |334      |4.2   |4.23  |2.63  |
|5             |0.31     |Good       |J        |SI2        |63.3     |58.0     |335      |4.34  |4.35  |2.75  |
|6             |0.24     |Very Good  |J        |VVS2       |62.8     |57.0     |336      |3.94  |3.96  |

NameError: name 'diamonds_df' is not defined